In [ ]:
import tensorflow as tf
import keras_cv
import matplotlib.pyplot as plt

from tqdm import tqdm

from dataset_utils import read_annotations, FOLDS, FOLDS_ROOT, IMAGES_ROOT
from inference_utils import (
    get_sliding_window_detections_with_scores, 
    get_sliding_window_inference_boxes, 
    get_sliding_window_detections,
    EMPTY_LABELS,
    format_labels
)

from plotting import plot_image_with_bounding_boxes, load_image_to_array

split_type = "VAL"
folds = FOLDS[split_type]

all_annotations_per_image = read_annotations(FOLDS_ROOT, folds)

model = tf.keras.models.load_model("simple_model.keras")

input_height = model.input.shape[1]
input_width = model.input.shape[2]

In [ ]:
for key in list(all_annotations_per_image.keys())[:2]:

    gt_bboxes = all_annotations_per_image[key]

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))
    plot_image_with_bounding_boxes(img_array, gt_bboxes)

In [ ]:
for key in list(all_annotations_per_image.keys())[:2]:

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))

    gt_bboxes = all_annotations_per_image[key]
    inference_boxes = get_sliding_window_inference_boxes(img_array.shape, (input_height, input_width))

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))
    plot_image_with_bounding_boxes(img_array, gt_bboxes, inference_boxes)

In [ ]:
for key in list(all_annotations_per_image.keys())[:10]:

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))

    gt_bboxes = all_annotations_per_image[key]
    detection_boxes = get_sliding_window_detections(img_array, (input_height, input_width), model)

    plot_image_with_bounding_boxes(img_array, gt_bboxes, detection_boxes)

In [ ]:
coco_metrics = keras_cv.metrics.BoxCOCOMetrics(
    bounding_box_format="xyxy", evaluate_freq=1
)


for key in tqdm(list(all_annotations_per_image.keys())):

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))

    labels = format_labels(all_annotations_per_image[key])

    if len(all_annotations_per_image[key]) == 0:
        labels = EMPTY_LABELS

    detection_result = get_sliding_window_detections_with_scores(img_array, (input_height, input_width), model)

    coco_metrics.update_state(labels, detection_result)

coco_metrics.result()


In [ ]:
for key in tqdm(list(all_annotations_per_image.keys())[:10]):
    coco_metrics = keras_cv.metrics.BoxCOCOMetrics(
        bounding_box_format="xyxy", evaluate_freq=1
    )

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))
    labels = format_labels(all_annotations_per_image[key])
    detection_boxes = get_sliding_window_detections(img_array, (input_height, input_width), model)
    plot_image_with_bounding_boxes(img_array, all_annotations_per_image[key], detection_boxes)

    if len(all_annotations_per_image[key]) == 0:
        labels = EMPTY_LABELS

    detection_result = get_sliding_window_detections_with_scores(img_array, (input_height, input_width), model)

    coco_metrics.update_state(labels, detection_result)
    print(coco_metrics.result()['MaP'])
    print(coco_metrics.result()['MaP@[IoU=50]'])
    print(coco_metrics.result()['MaP@[IoU=75]'])